In [9]:
from sklearn.model_selection import train_test_split
from __future__ import print_function
import numpy as np                 # to use numpy arrays
import tensorflow as tf            # to specify and run computation graphs
import tensorflow_datasets as tfds # to load training data
from tensorflow import keras
from tensorflow.keras import backend
from tensorflow.keras import Model, initializers, regularizers, constraints
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Bidirectional, Embedding, GlobalMaxPooling1D, Dense, Flatten, Conv2D, MaxPooling2D, MaxPool2D, Dropout, GlobalAvgPool2D, Input
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.layers import *

import math as m
import matplotlib.pyplot as plt    # to visualize data and draw plots
import numpy as np
import seaborn as sns
from math import sqrt
import pandas as pd

In [10]:
train_data, test_data = tfds.load(
    name="imdb_reviews", 
    split=('train', 'test'),
    as_supervised=False)

temp = train_data.as_numpy_iterator()
train_data_2 = []
train_label_2 = []
for it in temp:
    train_data_2.append(it['text'])
    train_label_2.append(it['label'])
    
temp = test_data.as_numpy_iterator()
test_data_2 = []
test_label_2 = []
for it in temp:
    test_data_2.append(it['text'])
    test_label_2.append(it['label'])

In [ ]:
# Tokenize the training data
vocab_size = 60000
maxlen = 250
encode_dim = 70
batch_size = 32

tokenizer = Tokenizer()

# Tokenize training data
for it in train_data_2:
    tokenizer.fit_on_texts(str(it))
    
tokenized_word_list_train_2 = []
for it in train_data_2:
    temp = tokenizer.texts_to_sequences(str(it))
    newL = []
    for it2 in temp:
        if it2 == []:
            continue
        newL.append(it2[0])
    tokenized_word_list_train_2.append(newL)
    
X_train_padded = pad_sequences(tokenized_word_list_train_2, maxlen = maxlen, padding='post')

In [ ]:
# Tokenize the test data

for it in test_data_2:
    tokenizer.fit_on_texts(str(it))

tokenized_word_list_test_2 = []
for it in test_data_2:
    temp = tokenizer.texts_to_sequences(str(it))
    newL = []
    for it2 in temp:
        if it2 == []:
            continue
        newL.append(it2[0])
    tokenized_word_list_test_2.append(newL)
    
X_test_padded = pad_sequences(tokenized_word_list_test_2, maxlen = maxlen, padding='post')

In [5]:
# Attention layer
class AttentionWithContext(tf.keras.layers.Layer):
    """
    Attention operation, with a context/query vector, for temporal data.
    Supports Masking.
    Follows the work of Yang et al. [https://www.cs.cmu.edu/~diyiy/docs/naacl16.pdf]
    "Hierarchical Attention Networks for Document Classification"
    by using a context vector to assist the attention
    # Input shape
        3D tensor with shape: `(samples, steps, features)`.
    # Output shape
        2D tensor with shape: `(samples, features)`.
    How to use:
    Just put it on top of an RNN Layer (GRU/LSTM/SimpleRNN) with return_sequences=True.
    The dimensions are inferred based on the output shape of the RNN.
    Note: The layer has been tested with Keras 2.0.6
    Example:
        model.add(LSTM(64, return_sequences=True))
        model.add(AttentionWithContext())
        # next add a Dense layer (for classification/regression) or whatever...
    """

    def __init__(self, W_regularizer=None, u_regularizer=None, b_regularizer=None,                
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs):

        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(AttentionWithContext, self).__init__(**kwargs)
    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight(shape = (input_shape[-1], input_shape[-1],),
                                 initializer=self.init,
                                 name = 'name',
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight(shape = (input_shape[-1],),
                                     initializer='zero',
                                     name = 'name',
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)

        self.u = self.add_weight(shape = (input_shape[-1],),
                                 initializer=self.init,
                                 name = 'name',
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)
        
        super(AttentionWithContext, self).build(input_shape)

    def compute_mask(self, input, input_mask=None):
        # do not pass the mask to the next layers
        return None

    def call(self, x, mask=None):
        uit = dot_product(x, self.W)

        if self.bias:
            uit += self.b

        uit = tf.math.tanh(uit)
        ait = dot_product(uit, self.u)

        a = tf.math.exp(ait)

        # apply mask after the exp. will be re-normalized next
        if mask is not None:
            # Cast the mask to floatX to avoid float64 upcasting in theano
            a *= tf.cast(mask, tf.float32)
        
        # in some cases especially in the early stages of training the sum may be almost zero
        # and this results in NaN's. A workaround is to add a very small positive number ε to the sum.
        # a /= K.cast(K.sum(a, axis=1, keepdims=True), K.floatx())
        a /= tf.cast(tf.math.reduce_sum(a, axis=1, keepdims=True) + backend.epsilon(), tf.float32)

        a = tf.expand_dims(a, -1)
        print(x.shape)
        print(a.shape)
        weighted_input = x * a
        return tf.math.reduce_sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]

def dot_product(x, kernel):
    """
    Wrapper for dot product operation, in order to be compatible with both
    Theano and Tensorflow
    Args:
        x (): input
        kernel (): weights
    Returns:
    """
    return tf.tensordot(x, kernel, axes = 1)

In [6]:
model = Sequential()
embed = Embedding(input_dim = vocab_size, output_dim = 20, input_length = X_train_padded.shape[1])

model.add(embed)
model.add(Dropout(0.4))
model.add(Bidirectional(GRU(256, return_sequences = True)))
model.add(Dropout(0.3))
model.add(Bidirectional(GRU(256, return_sequences = True)))
model.add(Dropout(0.3))
model.add(Bidirectional(GRU(256, return_sequences = True)))
model.add(Dropout(0.3))
model.add(AttentionWithContext())
model.add(Dropout(0.5))
model.add(Dense(200))
model.add(LeakyReLU(alpha=0.2))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
model.summary()

(None, 250, 512)
(None, 250, 1)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 250, 20)           1200000   
_________________________________________________________________
dropout (Dropout)            (None, 250, 20)           0         
_________________________________________________________________
bidirectional (Bidirectional (None, 250, 512)          427008    
_________________________________________________________________
dropout_1 (Dropout)          (None, 250, 512)          0         
_________________________________________________________________
bidirectional_1 (Bidirection (None, 250, 512)          1182720   
_________________________________________________________________
dropout_2 (Dropout)          (None, 250, 512)          0         
_________________________________________________________________
bidirectional_2 (Bidirec

In [7]:
# The inputs are actually X_train_padded, and the label is train_label_2
X_train_final2, X_val, y_train_final2, y_val = train_test_split(X_train_padded, train_label_2, test_size = 0.2, shuffle=True)

X_train_final2 = np.array(X_train_final2)
X_val = np.array(X_val)
y_train_final2 = np.array(y_train_final2)
y_val = np.array(y_val)

In [8]:
#Fitting the model
history = model.fit(X_train_final2, y_train_final2, epochs = 20 , batch_size = batch_size, verbose = 1, validation_data = (X_val, y_val))

Epoch 1/20
(32, 250, 512)
(32, 250, 1)
(32, 250, 512)
(32, 250, 1)
625/625 [==============================] - ETA: 0s - loss: 0.6952 - accuracy: 0.4990(None, 250, 512)
(None, 250, 1)
625/625 [==============================] - 310s 497ms/step - loss: 0.6952 - accuracy: 0.4990 - val_loss: 0.6943 - val_accuracy: 0.4998
Epoch 2/20
 34/625 [>.............................] - ETA: 4:20 - loss: 0.6933 - accuracy: 0.5156

KeyboardInterrupt: 

In [ ]:
print(history)

In [14]:
X_test_final2 = X_test_padded
Y_test_final2 = test_label_2

In [15]:
def print_confusion_matrix(validation_labels, y_pred):
    con_Matrix = tf.math.confusion_matrix(labels=validation_labels, predictions=y_pred).numpy()

    con_Matrix_norm = np.around(con_Matrix.astype('float') / con_Matrix.sum(axis=1)[:, np.newaxis], decimals=2)

    con_mat_df = pd.DataFrame(con_Matrix_norm, index = classes, columns = classes)


    #plots the heat map of the con matrix
    figure = plt.figure()
    sns.heatmap(con_mat_df, annot=True,cmap=plt.cm.Blues)
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.show()
    
def confidence_interval(validation_evaluation, validation_size):
    validation_data_error = 1 - validation_evaluation[1]
    print(validation_data_error)

    lower_bound_interval = validation_data_error - 1.96 * sqrt(
        (validation_data_error * (1 - validation_data_error)) / validation_size)
    upper_bound_interval = validation_data_error + 1.96 * sqrt(
        (validation_data_error * (1 - validation_data_error)) / validation_size)
    return (lower_bound_interval, upper_bound_interval)

def print_accuracy_graph(history):
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('model accuracy')
    plt.ylabel('accuracy')
    plt.xlabel('epoch')
    plt.legend(['train', 'val'], loc='upper left')
    plt.show()

def print_loss_graph(history):
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('model loss')

In [18]:
validation_evaluation = model.evaluate(X_test_final2, Y_test_final2) # Stores [loss, accuracy] of model

#all until functions
y_predict = model.predict(X_test_final2)
y_pred = model.predict_classes(X_test_final2) #for confusion matrix
print_confusion_matrix(Y_test_final2, y_pred)
print_accuracy_graph(history)
print_loss_graph(history)
confidence_interval(validation_evaluation, len(Y_test_final2))

157/157 [==============================] - 24s 150ms/step - loss: 0.5900 - accuracy: 0.6638


NameError: name 'pd' is not defined

In [ ]:
util_FMNIST.print_confusion_matrix(X_test_final2,y_pred)
util_FMNIST.print_accuracy_graph(history)
util_FMNIST.print_loss_graph(history)
lower_bound_interval, upper_bound_interval = util_FMNIST.confidence_interval(validation_evaluation, len(Y_test_final2))
print("The 95% Confidence interval for error hypothesis based on the normal distribution estimator: ",
      (lower_bound_interval, upper_bound_interval))


In [ ]:
tf.saved_model.save(model, 'homework1/FMNIST')